#### 추가 실습
- 공공API 데이터 끌어오기 (친숙해지기)
- (필수) 특정 월에 서울시 전체 구의 데이터를 한 번에 가져오기(입력 값: 년월 ex. 202402)
- (선택) 특정 서울시 하나의 구의 과거 1년치 데이터를 한 번에 가져오기(입력 값: 특정 구 하나)

작업 정리
1. 데이터 LOAD
2. 데이터 처리
3. 결과 데이터 파일 OUTPUT

In [18]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

In [3]:
# 공공 API 키 정보 로드

def load_key():
    api_key_file = open("pubAPI/my_api_key.txt","r")
    key_data=api_key_file.readlines()
    my_key=key_data[0]
    api_key_file.close()
    
    return my_key

In [282]:
#공공 API 전달 데이터 프리셋 생성

base_url = "http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade"

def setData(base_url,lawd_cd, yyyymm,API_key):
    
    area_cd = "LAWD_CD=" + lawd_cd
    deal_dt = "DEAL_YMD=" + yyyymm
    svc_key = "serviceKey=" + API_key
    result_url = base_url + "?" + area_cd + "&" + deal_dt + "&" + svc_key + "&" +"wadl&type=xml"
    
    return result_url

In [14]:
#print(setData(base_url,param, load_key()))

In [59]:
def find_xml_items(response):
    root = ET.fromstring(response.content)
    item_list = []
    for child in root.find('body').find('items'):
        elements = child.findall('*')
        data = {}
        for element in elements:
            tag = element.tag.strip()
            text = element.text.strip()
            data[tag] = text
        item_list.append(data)  
    return item_list

In [52]:
# 법정동명, 코드 매핑

def getCode_lawd(lawd_name):
    lawd_cd = pd.read_csv("pubAPI/법정동코드 전체자료.txt", sep='\t', encoding ='cp949') #'euc-kr', 'utf-8'
    
    return lawd_cd[(lawd_cd['법정동명'].str.contains(lawd_name)) & (lawd_cd['폐지여부'] == "존재")].head(1).iloc[0,0].astype(str)[0:5]
    

In [55]:
#공공 API 호출 및 저장

def getData(url,lawd_name, yyyymm):
    response = requests.get(url)
    items_list = find_xml_items(response)
    result = pd.DataFrame(items_list)
    
    name_info = lawd_name + "_" + yyyymm
    file_name = f"pubAPI/result_{name_info}.csv"

    result.to_csv(file_name, index=False, mode='a')
    
    return file_name

<Response [200]>


'pubAPI/result_종로구_202301.csv'

In [357]:
#### - Input Data : LAWD_CD, DEAL_YMD, serviceKey  입력받기
in_lawd = input("검색지역 입력 : ")

in_deal_ymd = input("계약월 입력(예: 201911) : ")

getData(setData(base_url,getCode_lawd(in_lawd), in_deal_ymd, load_key()), in_lawd, in_deal_ymd)

KeyboardInterrupt: Interrupted by user

(필수) 특정 월에 서울시 전체 구의 데이터를 한 번에 가져오기(입력 값: 년월 ex. 202402)
(선택) 특정 서울시 하나의 구의 과거 1년치 데이터를 한 번에 가져오기(입력 값: 특정 구 하나)

In [347]:
def listing_lawd_code(str_):
    
    temp = []
    temp1 = []
    lawd_cd = pd.read_csv("pubAPI/법정동코드 전체자료.txt", sep='\t', encoding ='cp949') #'euc-kr', 'utf-8'

    seoul_indices = lawd_cd.index[lawd_cd['법정동명'].str.contains(str_)]
    
    for i in range(len(seoul_indices)):
        temp.append(lawd_cd.iloc[i]['법정동코드'].astype(str)[0:5])
        temp1
    
    temp_set = set(temp)
    temp_set1 = set(temp1)    
    
    temp_list1 = list(temp_set1)
    
    return list(temp_set)

In [359]:
def listing_lawd_name(str_):
    
    temp = []
    result = []
    lawd_cd = pd.read_csv("pubAPI/법정동코드 전체자료.txt", sep='\t', encoding ='cp949') #'euc-kr', 'utf-8'

    seoul_indices = lawd_cd.loc[(lawd_cd['법정동명'].str.contains(str_)) & (lawd_cd['폐지여부'] == "존재"),'법정동명']
    
    for i in seoul_indices:
        temp = i.split()
        if(len(temp) > 2):
            result.append(temp[1])
        
    return list(set(result))


In [360]:
listing_lawd_name("서울특별시")

['노원구',
 '중랑구',
 '성북구',
 '동작구',
 '동대문구',
 '마포구',
 '종로구',
 '송파구',
 '은평구',
 '양천구',
 '성동구',
 '금천구',
 '관악구',
 '영등포구',
 '강북구',
 '강남구',
 '용산구',
 '서대문구',
 '강서구',
 '도봉구',
 '강동구',
 '구로구',
 '광진구',
 '중구',
 '서초구']

In [358]:
#(필수) 특정 월에 서울시 전체 구의 데이터를 한 번에 가져오기(입력 값: 년월 ex. 202402)

date = input("계약월 입력(예: 202402) : ")

for i in listing_lawd_name("서울특별시"):
    getData(setData(base_url, i, date, load_key()), i, date)


계약월 입력(예: 202402) : 202402


AttributeError: 'NoneType' object has no attribute 'find'

In [ ]:
type(listing_lawd())

In [ ]:
#(선택) 특정 서울시 하나의 구의 과거 1년치 데이터를 한 번에 가져오기(입력 값: 특정 구 하나)

In [366]:
from datetime import datetime, timedelta

def get_past_year_months(input_month):
    # 입력된 월을 YYYYMM 형식으로 변환
    input_date = datetime.strptime(input_month, "%Y%m")

    # 과거 1년치의 월 리스트 생성
    past_year_months = []
    for i in range(12):
        month = input_date - timedelta(days=30*(i+1))
        past_year_months.append(month.strftime("%Y%m"))

    return past_year_months[::-1]  # 리스트를 역순으로 반환하여 최근 월부터 순서대로 출력


# 결과 출력
print("과거 1년치 월 리스트:", get_past_year_months(input_month))

YYYYMM 형식으로 월을 입력하세요: 


ValueError: time data '' does not match format '%Y%m'

YYYYMM 형식으로 월을 입력하세요: 202302


['202202',
 '202203',
 '202204',
 '202205',
 '202206',
 '202207',
 '202208',
 '202209',
 '202210',
 '202211',
 '202212',
 '202301']

In [368]:
in_lawd = input("검색지역 입력 : ")

for i in get_past_year_months(datetime.now().strftime("%Y%m")):
    getData(setData(base_url,getCode_lawd(in_lawd), i, load_key()), in_lawd, i)

검색지역 입력 : 종로구


AttributeError: 'NoneType' object has no attribute 'find'